Testing the experiment

In [1]:
from experiments.experiment import *

results = Experiment.load_results()

In [3]:
for k,v in results.items():
    print(v)
    break


   Unnamed: 0  amount virgin  amount recyclate high  amount recyclate low  \
0           0    1779.711679             725.935917            726.352403   
1           0    1775.049353             726.389207            726.561439   

   amount reused parts  amount standard parts  amount leakage  price virgin  \
0                 2666                    566       13.811654      2.535715   
1                 2675                    553       13.974876      2.569072   

   price recyclate  
0         2.493949  
1         2.337574  
